# LEE - Latent Entities Extraction
# Multitask Learning with Task-Grouping model

In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline
DATA_PATH = './data'

## Import preprocessed data

In [3]:
train_df = pd.read_csv(os.path.join(DATA_PATH, 'preprocessed_train.csv'))
dev_df = pd.read_csv(os.path.join(DATA_PATH, 'preprocessed_dev.csv'))
test_df = pd.read_csv(os.path.join(DATA_PATH, 'preprocessed_test.csv'))

In [4]:
for curr_df in[train_df, dev_df, test_df]:
    curr_df.reactants = curr_df.reactants.apply(eval)
    curr_df.products = curr_df.products.apply(eval)
    curr_df.entities = curr_df.entities.apply(eval)

## Model

### Defining pre-defined entities list to extract

In [5]:
entities_histogram = pd.Series([e for ents in train_df.entities for e in ents]).value_counts()

In [6]:
NUM_CLASSES = 40
classes = list(entities_histogram.head(NUM_CLASSES).index)

In [7]:
print(classes)

['atp', 'adp', 'h2o', 'pi', 'h+', 'o2', 'nadph', 'nadp+', 'coa-sh', 'ppi', 'gtp', 'gdp', 'adomet', 'ub', 'adohcy', 'co2', 'nad+', 'amp', 'nadh', '2og', 'na+', 'l-glu', 'pi(4,5)p2', 'ac-coa', 'ube2i', 'pi(3,4,5)p3', 'lcfa(-)', 'h2o2', 'udp', 'succa', 'p-s15,s20-tp53 tetramer', 'p21 ras:gtp', 'ca2+', 'p21 ras:gdp', 'sumo1:c93-ube2i', 'grb2-1:sos1', 'dag', 'runx2', 'pap', 'ch2o']


## Load word embedding
Source & Pre-trained embeddings: https://github.com/cambridgeltl/BioNLP-2016

In [8]:
import gensim
w2v = gensim.models.KeyedVectors.load_word2vec_format(os.path.join(DATA_PATH, 'bio_nlp_vec/PubMed-shuffle-win-30.bin'), binary=True)

In [9]:
from keras.preprocessing.text import text_to_word_sequence, hashing_trick
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


### Effective vocabulary

In [10]:
vocab = [text_to_word_sequence(n) for n in train_df.notes.values]
vocab = {x for l in vocab for x in l}
print('Vocabulary size:', len(vocab))

Vocabulary size: 24244


### Padding Sequences & Handilng UNKNOWN tokens

In [11]:
word2idx = {}
embeddings = []
word2idx["PADDING_TOKEN"] = len(word2idx)
vector = np.zeros(w2v.vector_size) #Zero vector vor 'PADDING' word
embeddings.append(vector)

word2idx["UNKNOWN_TOKEN"] = len(word2idx) - 1
vector = np.random.uniform(-0.25, 0.25, w2v.vector_size)
embeddings.append(vector)

In [12]:
for w in w2v.vocab:
    word = w.lower()
    if word in word2idx or word not in vocab:
        continue
        
    word2idx[word] = len(word2idx) - 1
    embeddings.append(w2v[w])

In [13]:
embeddings = np.array(embeddings)

### Hashing for sequences generation

In [14]:
def hash_word(w):
    try:
        return word2idx[w]
    except KeyError:
        return word2idx['UNKNOWN_TOKEN']

### Convert textual passages into a embedding sequences

In [15]:
vocab_size = len(word2idx)

In [16]:
encoded_one_hot = [hashing_trick(d, vocab_size, hash_function=hash_word) for d in train_df.notes.values]

In [17]:
max_len_seq = max([len(x) for x in encoded_one_hot])

In [18]:
padded_notes = pad_sequences(encoded_one_hot, maxlen=max_len_seq, padding='post')

In [19]:
def extract_seq(curr_df, max_len=max_len_seq):
    vocab_size = len(word2idx)
    encoded_one_hot = [hashing_trick(d, vocab_size, hash_function=hash_word) for d in curr_df.notes.values]
    padded_notes = pad_sequences(encoded_one_hot, maxlen=max_len, padding='post')
    return padded_notes

### Build Co-Occurence Matrix for Task-Grouping

In [20]:
occurences = train_df.entities.apply(lambda e: [1 if c in e else 0 for c in classes])
occurences = np.array([np.array(o) for o in occurences])

In [21]:
cooccurrence_matrix = occurences.T.dot(occurences)

In [22]:
import scipy.sparse as sp
g = sp.diags(1./cooccurrence_matrix.diagonal())
normalized_cooccurence = g * cooccurrence_matrix # normalized co-occurence matrix

In [23]:
class UnionFind:
    """Weighted quick-union with path compression.
    The original Java implementation is introduced at
    https://www.cs.princeton.edu/~rs/AlgsDS07/01UnionFind.pdf
    """

    def __init__(self, n):
        self._id = list(range(n))
        self._sz = [1] * n

    def _root(self, i):
        j = i
        while (j != self._id[j]):
            self._id[j] = self._id[self._id[j]]
            j = self._id[j]
        return j

    def find(self, p, q):
        return self._root(p) == self._root(q)

    def union(self, p, q):
        i = self._root(p)
        j = self._root(q)
        if (self._sz[i] < self._sz[j]):
            self._id[i] = j
            self._sz[j] += self._sz[i]
        else:
            self._id[j] = i
            self._sz[i] += self._sz[j]

In [25]:
uf = UnionFind(NUM_CLASSES)
threshold = 0.65
for i1,c1 in enumerate(classes):
    for i2,c2 in enumerate(classes):
        if normalized_cooccurence[i1,i2] > threshold or normalized_cooccurence[i2,i1] > threshold:
            uf.union(i1, i2)

In [26]:
for i in range(NUM_CLASSES):
    count_together = sum([1 if uf.find(i, j) else 0 for j in range(NUM_CLASSES)]) - 1
    if count_together == 0 and sorted(normalized_cooccurence[i], reverse=True)[1] > threshold / 2:
        uf.union(i, normalized_cooccurence[i].argsort()[::-1][1])

In [27]:
d = {c: classes[uf._root(classes.index(c))] for c in classes}

In [28]:
v = {}
for key, value in sorted(d.items()):
    v.setdefault(value, []).append(key)

In [29]:
for x in v.values():
    print(x)

['2og', 'ch2o', 'co2', 'h2o2', 'o2', 'succa']
['ac-coa', 'coa-sh']
['adohcy', 'adomet']
['adp', 'amp', 'atp', 'pi(3,4,5)p3', 'pi(4,5)p2', 'ppi']
['ca2+']
['dag', 'h2o', 'lcfa(-)', 'pi']
['gdp', 'gtp', 'p21 ras:gdp', 'p21 ras:gtp']
['grb2-1:sos1']
['h+', 'nad+', 'nadh', 'nadp+', 'nadph']
['l-glu']
['na+']
['p-s15,s20-tp53 tetramer']
['pap']
['runx2']
['sumo1:c93-ube2i', 'ube2i']
['ub']
['udp']


In [30]:
tasks = list(v.values())

In [31]:
print('Task groups are:')
for x in tasks:
    print(x)

Task groups are:
['2og', 'ch2o', 'co2', 'h2o2', 'o2', 'succa']
['ac-coa', 'coa-sh']
['adohcy', 'adomet']
['adp', 'amp', 'atp', 'pi(3,4,5)p3', 'pi(4,5)p2', 'ppi']
['ca2+']
['dag', 'h2o', 'lcfa(-)', 'pi']
['gdp', 'gtp', 'p21 ras:gdp', 'p21 ras:gtp']
['grb2-1:sos1']
['h+', 'nad+', 'nadh', 'nadp+', 'nadph']
['l-glu']
['na+']
['p-s15,s20-tp53 tetramer']
['pap']
['runx2']
['sumo1:c93-ube2i', 'ube2i']
['ub']
['udp']


In [32]:
print('Number of task-groups:', len(tasks))

Number of task-groups: 17


### Define X & Y for training, validating and testing

In [33]:
def target_classes(dataframe):
    def single_entry(entry, task_entities):
        return np.array([1 if c in entry else 0 for c in task_entities])

    def single_task(task_entities):
        return np.array([single_entry(entry, task_entities) for entry in dataframe.entities])
    
    return [single_task(task) for task in tasks]
    

In [34]:
train_x, dev_x, test_x = extract_seq(train_df), extract_seq(dev_df), extract_seq(test_df)
train_y, dev_y, test_y = target_classes(train_df), target_classes(dev_df), target_classes(test_df)

### Model Definition

In [35]:
from keras import backend as K

def mcor(y_true, y_pred):
     #matthews_correlation
     y_pred_pos = K.round(K.clip(y_pred, 0, 1))
     y_pred_neg = 1 - y_pred_pos
 
     y_pos = K.round(K.clip(y_true, 0, 1))
     y_neg = 1 - y_pos
 
     tp = K.sum(y_pos * y_pred_pos)
     tn = K.sum(y_neg * y_pred_neg)
 
     fp = K.sum(y_neg * y_pred_pos)
     fn = K.sum(y_pos * y_pred_neg)
 
 
     numerator = (tp * tn - fp * fn)
     denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
     return numerator / (denominator + K.epsilon())

def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))


In [36]:
def multitask_loss(y_true, y_pred):
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    return K.mean(K.sum(- y_true * K.log(y_pred) - (1 - y_true) * K.log(1 - y_pred), axis=1))

In [37]:
from keras.layers import Dense, Embedding, Bidirectional, GRU, Input
from keras.models import Model

In [38]:
def model_mtl_bi_gru_task_grouping():
    inputs = Input(shape=(max_len_seq,), dtype='int32')
    embedded = Embedding(input_length=max_len_seq, input_dim=embeddings.shape[0],
                         output_dim=embeddings.shape[1],
                         weights=[embeddings], trainable=False)(inputs)
    lstm_out = Bidirectional(
        GRU(units=200, activation='sigmoid', dropout=0.50, recurrent_dropout=0.25, return_sequences=False))(
        embedded)

    output_tasks = [Dense(len(task), activation='sigmoid', name='layer_'+str(i))(lstm_out) for i, task in enumerate(tasks)]

    model = Model(inputs=[inputs], outputs=output_tasks)
    model.compile(loss=[multitask_loss] * len(tasks), optimizer= "adam", metrics=['accuracy', f1, precision, recall, mcor])
    model.summary()
    return model


### Training

In [39]:
current_model = model_mtl_bi_gru_task_grouping

In [40]:
model = current_model()
model_name = current_model.__name__
print('number of layers:', len(model.layers))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 830)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 830, 200)     4325000     input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 400)          481200      embedding_1[0][0]                
__________________________________________________________________________________________________
layer_0 (Dense)                 (None, 6)            2406        bidirectional_1[0][0]            
__________________________________________________________________________________________________
layer_1 (D

#### Training

In [ ]:
from keras.callbacks import ModelCheckpoint
print('CURRENT MODEL:', model_name)
filepath = model_name + "_weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
model.fit(train_x, train_y, epochs=350, batch_size=128, callbacks=callbacks_list, validation_split=0.2)

#### OPTIONAL: Load weights for trained model
Note that as a result of float-numbers precision errors, the calculated co-occurence matrix may be differed in different executions. 
Therefore, in case you train and then evaluate in two different stages (different machines/sessions), make sure that the task-group division is equivalent in both of them.

In [ ]:
# model.load_weights('filename')

### Evaluation

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score, precision_score, recall_score, f1_score, accuracy_score, precision_recall_curve, auc

In [ ]:
y_true = target_classes(test_df)
y_proba = model.predict(test_x)

In [ ]:
y_true = np.concatenate(y_true, axis=1)
y_proba = np.concatenate(y_proba, axis=1)

In [ ]:
threshold = 0.5
y_predict = np.where(y_proba > threshold, 1, 0)

In [ ]:
results = []
for avg_metric in ['micro', 'macro']:
    p = precision_score(y_true, y_predict , average=avg_metric)
    r = recall_score(y_true, y_predict , average=avg_metric)
    f1r = f1_score(y_true, y_predict , average=avg_metric)
    results += [avg_metric, p, r, f1r]

In [ ]:
results

In [ ]:
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()


for i in range(len(classes)):
    fpr[i], tpr[i], _ = roc_curve(y_true[:, i], y_proba[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_true.ravel(), y_proba.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
print(roc_auc["micro"])

In [ ]:
from scipy import interp
# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(NUM_CLASSES)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(NUM_CLASSES):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= NUM_CLASSES

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
print(roc_auc["macro"])


In [ ]:
# Plot all ROC curves
plt.figure()
lw = 2
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.3f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.3f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()